In [1]:
# import libraries
import numpy as np
import pandas as pd
import random
import csv
import scipy.stats as stats
import seaborn as sns
from collections import Counter
from collections import defaultdict
import math
from sklearn.linear_model import ElasticNet
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from random import randint

import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.nonparametric import smoothers_lowess
from pandas import Series, DataFrame
from patsy import dmatrices
from sklearn import datasets, svm

# initialize the plotting sizes
# set size
plt.rc('figure', figsize=(15, 8))
# subplots size
fizsize_with_subplots = (15, 8)
# histogram size
bin_size = 10

In [2]:
# Loading data
dates = pd.read_csv('data/dates.csv',header=None)
queries = pd.read_csv('data/queries.csv',header=None)
X = pd.read_csv('data/X.csv',header=None)
y = pd.read_csv('data/y.csv',header=None)

X.columns = queries

In [3]:
# data preprocessing and train-test split
# this data is well formed with no missing value and other symbols or labels that are non numerical.
# splitting the data into train and test

# Here we want the first 500 queries, ARIAMA only consider y, setting X has no effect
X = X.iloc[:, 0: 500]

# first with the last year as test
# use the last 365 of training set as validation set
test_size =365
val_size=365
length = X.shape[0]
train1_X = X[0:length-test_size*2]
val1_X = X[length-test_size*2:length-test_size]
train1_y = y[0:length-test_size*2]
val1_y = y[length-test_size*2:length-test_size]
test1_X = X[length-test_size:]
test1_y = y[length-test_size:]

#without validation set
train1_X2 = X[0:length-test_size]
train1_y2 = y[0:length-test_size]

# second with the last 2 year as test
test_size2 =365*2
train2_X = X[0:length-test_size2-val_size]
val2_X = X[length-test_size2-val_size:length-test_size2]
train2_y = y[0:length-test_size2-val_size]
val2_y = y[length-test_size2-val_size:length-test_size2]
test2_X = X[length-test_size2:]
test2_y = y[length-test_size2:]

#without validation set
train2_X2 = X[0:length-test_size2]
train2_y2 = y[0:length-test_size2]


print('X shape: ',X.shape,'  y shape: ',y.shape)
print('train1 X:',train1_X.shape,' train1 y:',train1_y.shape,' Test1 X shape:',test1_X.shape, ' Test1 y shape:',test1_y.shape)
print('train2 X:',train2_X.shape,' train2 y:',train2_y.shape,' Test2 X:',test2_X.shape,' Test2 y:',test2_y.shape)

X shape:  (4383, 500)   y shape:  (4383, 1)
train1 X: (3653, 500)  train1 y: (3653, 1)  Test1 X shape: (365, 500)  Test1 y shape: (365, 1)
train2 X: (3288, 500)  train2 y: (3288, 1)  Test2 X: (730, 500)  Test2 y: (730, 1)


In [4]:
# using first 4 years predicting the 5th, then first 5 predict 6th, until first 10 predicting 11th, 
# then all together as exogenous variable of arimax predicting test set
alpha = 11
l1_ratio = 0.3
X_train = train1_X2[0:4*365]
y_train= train1_y2[0:4*365]
X_test = train1_X2[4*365:5*365]
enet = ElasticNet(alpha=11, l1_ratio=0.3, normalize=False,max_iter=10000)
enet.fit(X_train,y_train)
print('Nonzero weights: %d from %d' % (len(np.nonzero(enet.coef_)[0]), len(enet.coef_)))
y_pred5 = enet.predict(X_test)

Nonzero weights: 170 from 500


In [5]:
X_train = train1_X2[365:5*365]
y_train= train1_y2[365:5*365]
X_test = train1_X2[5*365:6*365]
enet = ElasticNet(alpha=11, l1_ratio=0.3, normalize=False,max_iter=10000)
enet.fit(X_train,y_train)
print('Nonzero weights: %d from %d' % (len(np.nonzero(enet.coef_)[0]), len(enet.coef_)))
y_pred6 = enet.predict(X_test)

Nonzero weights: 138 from 500


In [6]:
X_train = train1_X2[2*365:6*365]
y_train= train1_y2[2*365:6*365]
X_test = train1_X2[6*365:7*365]
enet = ElasticNet(alpha=11, l1_ratio=0.3, normalize=False,max_iter=10000)
enet.fit(X_train,y_train)
print('Nonzero weights: %d from %d' % (len(np.nonzero(enet.coef_)[0]), len(enet.coef_)))
y_pred7 = enet.predict(X_test)

Nonzero weights: 155 from 500


In [7]:
X_train = train1_X2[3*365:7*365]
y_train= train1_y2[3*365:7*365]
X_test = train1_X2[7*365:8*365]
enet = ElasticNet(alpha=11, l1_ratio=0.3, normalize=False,max_iter=10000)
enet.fit(X_train,y_train)
print('Nonzero weights: %d from %d' % (len(np.nonzero(enet.coef_)[0]), len(enet.coef_)))
y_pred8 = enet.predict(X_test)

Nonzero weights: 146 from 500


In [8]:
X_train = train1_X2[4*365:8*365]
y_train= train1_y2[4*365:8*365]
X_test = train1_X2[8*365:9*365]
enet = ElasticNet(alpha=11, l1_ratio=0.3, normalize=False,max_iter=10000)
enet.fit(X_train,y_train)
print('Nonzero weights: %d from %d' % (len(np.nonzero(enet.coef_)[0]), len(enet.coef_)))
y_pred9 = enet.predict(X_test)

Nonzero weights: 96 from 500


In [10]:
X_train = train1_X2[5*365:9*365]
y_train= train1_y2[5*365:9*365]
X_test = train1_X2[9*365:10*365]
enet = ElasticNet(alpha=11, l1_ratio=0.3, normalize=False,max_iter=10000)
enet.fit(X_train,y_train)
print('Nonzero weights: %d from %d' % (len(np.nonzero(enet.coef_)[0]), len(enet.coef_)))
y_pred10 = enet.predict(X_test)

Nonzero weights: 62 from 500


In [11]:
X_train = train1_X2[6*365:10*365]
y_train= train1_y2[6*365:10*365]
X_test = train1_X2[10*365:]
enet = ElasticNet(alpha=11, l1_ratio=0.3, normalize=False,max_iter=10000)
enet.fit(X_train,y_train)
print('Nonzero weights: %d from %d' % (len(np.nonzero(enet.coef_)[0]), len(enet.coef_)))
y_pred11 = enet.predict(X_test)

Nonzero weights: 29 from 500


In [13]:
X_train = train1_X2[7*365:4018]
y_train= train1_y2[7*365:4018]
X_test = test1_X
enet = ElasticNet(alpha=11, l1_ratio=0.3, normalize=False,max_iter=10000)
enet.fit(X_train,y_train)
print('Nonzero weights: %d from %d' % (len(np.nonzero(enet.coef_)[0]), len(enet.coef_)))
y_pred12 = enet.predict(X_test)

Nonzero weights: 29 from 500


In [14]:
pred = np.concatenate((y_pred5, y_pred6,y_pred7,y_pred8,y_pred9,y_pred10,y_pred11,y_pred12))

In [15]:
pred.shape

(2923,)

In [16]:
EN_x = pd.DataFrame(pred)

In [17]:
EN_x.head()

,0
0,16.730904
1,10.894386
2,19.499658
3,15.090033
4,10.817456


In [18]:
EN_x.to_csv('EN_exog_new.csv')